To do: remove coins that have lack of history.  Many do not exist before 2021.

In [1]:
import portfolio_utils as pu
import constants as c

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

from tqdm import tqdm

pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)

In [2]:
symbols = ["BTCUSDT","ETHUSDT"]

#Select Rebalance Dates in constants file


#Portfolio Start and Stop Dates
start="2018-01-01" 
endp1="2021-11-01"

# some nice code curtesy of Kevin
agg_data = list()

crypto = pu.get_data(symbols, c.stems_btc_eth)#pd.read_csv(data_path)#, index_col="tradeDate")

crypto.tail()

,Open time,Open,High,Low,Close,Volume,Close time,Base asset volume,Number of trades,Taker buy volume,Taker buy base asset volume,Ignore,symbol,Open_time_,Close_time_
26,1635292800000,4131.52,4307.00,3909.00,3922.00,704604.25000,1635379199999,2.874313e+09,1872190,342946.25220,1.399582e+09,0,ETHUSDT,2021-10-27,2021-10-27 23:59:59.999
27,1635379200000,3921.99,4295.00,3890.17,4286.02,626378.50550,1635465599999,2.587268e+09,1711575,331721.67970,1.370288e+09,0,ETHUSDT,2021-10-28,2021-10-28 23:59:59.999
28,1635465600000,4286.01,4460.47,4265.97,4418.89,559231.07691,1635551999999,2.440933e+09,1799386,299827.57771,1.309029e+09,0,ETHUSDT,2021-10-29,2021-10-29 23:59:59.999
29,1635552000000,4418.89,4434.06,4232.20,4322.68,340223.06900,1635638399999,1.469665e+09,1035476,164985.54250,7.127256e+08,0,ETHUSDT,2021-10-30,2021-10-30 23:59:59.999
30,1635638400000,4322.68,4399.14,4164.35,4287.21,412856.28930,1635724799999,1.764887e+09,1303438,207029.19300,8.855724e+08,0,ETHUSDT,2021-10-31,2021-10-31 23:59:59.999


In [3]:
symbols

['BTCUSDT', 'ETHUSDT']

In [4]:
crypto.shape

(2800, 15)

In [5]:
crypto_ = crypto[["Close_time_", "symbol", "Close"]] #crypto_ = crypto[["tradeDate", "symbol", "avg_latest"]]
crypto_.columns=["tradeDate", "symbol", "Close"]
crypto_ = crypto_.pivot_table(index="tradeDate", columns="symbol", values = "Close")
crypto2_ = crypto_.pct_change()
crypto2_ = crypto2_[1:]
crypto2_.head()

symbol,BTCUSDT,ETHUSDT
tradeDate,,
2018-01-02 23:59:59.999,0.096794,0.132836
2018-01-03 23:59:59.999,0.016654,0.092075
2018-01-04 23:59:59.999,0.009386,0.006392
2018-01-05 23:59:59.999,0.126222,0.020532
2018-01-06 23:59:59.999,0.006450,0.046711


In [ ]:
sim_sum=10000
initial_investment = 10000
#potfolio_SR = np.zeros(sim_sum)
a = np.zeros(((sim_sum,len(symbols))))
portfolio_df = pd.DataFrame(a)
portfolio_df.columns = symbols
portfolio_df["Portfolio_SR"] = -999

for i in tqdm(range(sim_sum)):
    symbol_count = random.randint(2,len(symbols))
    symbol_sublist = random.sample(symbols, symbol_count)
    portfolio_weights = np.random.dirichlet(np.ones(symbol_count))

    df = crypto2_[symbol_sublist]
    clean_return_history = df.reset_index(drop=True)
    clean_return_history['date']=pu.date_cleaner(pd.Series(crypto2_.index.values).astype(str))
    
    # Instantiate the portfolio object
    portfolio_obj = pu.Portfolio(
        data=clean_return_history, 
        symbols=symbol_sublist, 
        weights=portfolio_weights, 
        initial_investment=initial_investment,
        start_date=start,
        end_date=endp1, 
        rebalance_dates=c.rebalance_dates)
    # estimate returns
    portfolio_obj.estimate_portfolio_returns()
    #print(portfolio_obj.get_sharpe_report())
    portfolio_df.loc[i,symbol_sublist] = portfolio_weights
    portfolio_df.loc[i,"Portfolio_SR"] = portfolio_obj.get_sharpe_ratio()
portfolio_df.to_csv(
    "data/random_portfolios_"+min(crypto.Close_time_.astype(str))+"_to_"+max(crypto.Close_time_.astype(str))+".csv", index=False)

 53%|█████▎    | 5299/10000 [52:30<48:09,  1.63it/s]  

In [ ]:
# equal weight portfolio
portfolio_weights = np.ones(len(symbols))/len(symbols)

df = crypto2_[symbols]
clean_return_history = df.reset_index(drop=True)
clean_return_history['date']=pu.date_cleaner(pd.Series(crypto2_.index.values).astype(str))

# Instantiate the portfolio object
portfolio_obj = pu.Portfolio(
    data=clean_return_history, 
    symbols=symbols, 
    weights=portfolio_weights, 
    initial_investment=initial_investment,
    start_date=start,
    end_date=endp1, 
    rebalance_dates=c.rebalance_dates)
# estimate returns
portfolio_obj.estimate_portfolio_returns()
#print(portfolio_obj.get_sharpe_report())

equal_wt_SR = portfolio_obj.get_sharpe_ratio()
equal_wt_SR

In [ ]:
#clean_return_history
ind_SR = np.sqrt(365)*np.mean(crypto2_, axis=0)/np.std(crypto2_, axis=0)
_=plt.hist(portfolio_df["Portfolio_SR"], label="Portfolio SR's", density=True)
plt.vlines(x=ind_SR[symbols],ymin=0, ymax=.1, color="red", label="individual SR's")
plt.axvline(equal_wt_SR, label="equal weight portfolio", color='black')
plt.axvline(2.04, label="S&P 500",color='pink')

_=plt.legend()
plt.suptitle("Portfolio Sharpe Ratios")
plt.title("BTCUSDT, ETHUSDT")

In [ ]:
portfolio_df.loc[portfolio_df.Portfolio_SR>max(ind_SR[symbols]),:].shape

In [ ]:
portfolio_df.sort_values(by=['Portfolio_SR'],ascending=False).head(500)

In [ ]:
crypto2_.head()